In [1]:
from PIL import Image

X = "HCKK0512.JPEG"

In [2]:
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()

exif = get_exif(X)
print(exif)

{36864: b'0231', 37121: b'\x01\x02\x03\x00', 37377: (84572, 17235), 36867: '2019:12:01 13:10:40', 36868: '2019:12:01 13:10:40', 37378: (193685, 85136), 37379: (53141, 12766), 37380: (0, 1), 37383: 5, 37385: 24, 37386: (83, 20), 40961: 1, 40962: 4032, 41989: 29, 41990: 0, 271: 'Apple', 272: 'iPhone 6s', 36880: '-08:00', 274: 1, 531: 1, 36881: '-08:00', 36882: '-08:00', 37521: '996', 37396: (2015, 1511, 2217, 1330), 37522: '996', 40963: 3024, 282: (72, 1), 283: (72, 1), 41495: 2, 33434: (1, 30), 33437: (11, 5), 41729: b'\x01', 34850: 2, 34853: {1: 'N', 2: ((49, 1), (16, 1), (3728, 100)), 3: 'W', 4: ((122, 1), (55, 1), (238, 100)), 5: b'\x00', 6: (321392, 1093), 12: 'K', 13: (0, 1), 16: 'T', 17: (323611, 1439), 23: 'T', 24: (323611, 1439), 29: '2019:12:01', 31: (759748, 331)}, 34855: 40, 296: 2, 41986: 0, 40960: b'0100', 41987: 0, 305: '13.2.3', 306: '2019:12:01 13:10:40', 42034: ((83, 20), (83, 20), (11, 5), (11, 5)), 42035: 'Apple', 42036: 'iPhone 6s back camera 4.15mm f/2.2', 34665: 20

In [3]:
from PIL.ExifTags import TAGS

def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val

    return labeled

exif = get_exif(X)
labeled = get_labeled_exif(exif)
print(labeled)

{'ExifVersion': b'0231', 'ComponentsConfiguration': b'\x01\x02\x03\x00', 'ShutterSpeedValue': (84572, 17235), 'DateTimeOriginal': '2019:12:01 13:10:40', 'DateTimeDigitized': '2019:12:01 13:10:40', 'ApertureValue': (193685, 85136), 'BrightnessValue': (53141, 12766), 'ExposureBiasValue': (0, 1), 'MeteringMode': 5, 'Flash': 24, 'FocalLength': (83, 20), 'ColorSpace': 1, 'ExifImageWidth': 4032, 'FocalLengthIn35mmFilm': 29, 'SceneCaptureType': 0, 'Make': 'Apple', 'Model': 'iPhone 6s', None: '-08:00', 'Orientation': 1, 'YCbCrPositioning': 1, 'SubsecTimeOriginal': '996', 'SubjectLocation': (2015, 1511, 2217, 1330), 'SubsecTimeDigitized': '996', 'ExifImageHeight': 3024, 'XResolution': (72, 1), 'YResolution': (72, 1), 'SensingMethod': 2, 'ExposureTime': (1, 30), 'FNumber': (11, 5), 'SceneType': b'\x01', 'ExposureProgram': 2, 'GPSInfo': {1: 'N', 2: ((49, 1), (16, 1), (3728, 100)), 3: 'W', 4: ((122, 1), (55, 1), (238, 100)), 5: b'\x00', 6: (321392, 1093), 12: 'K', 13: (0, 1), 16: 'T', 17: (323611,

In [4]:
from PIL.ExifTags import GPSTAGS

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

exif = get_exif(X)
geotags = get_geotagging(exif)
#print(geotags)-122.90839

In [5]:
def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])
    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)
def get_time(geotags):
    time = geotags['GPSDateStamp']
    return(time)

In [6]:
exif = get_exif(X)
geotags = get_geotagging(exif)
print(get_coordinates(geotags))

(49.27702, -122.91733)


In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
from math import cos, asin, sqrt, pi
data = pd.read_csv("photo.csv")
data

,name,lat,lon
0,HCKK0512,49.27702,-122.91733
1,IMG_0046,49.27817,-122.91140
2,IMG_0047,49.27812,-122.91081
3,IMG_0048,49.27802,-122.91026
4,IMG_0049,49.27802,-122.91013
5,IMG_0050,49.27564,-122.90968
6,IMG_0051,49.27551,-122.91369
7,IMG_0052,49.27584,-122.91425
8,IMG_0053,49.27648,-122.91597
9,IMG_0054,49.27655,-122.91631


In [8]:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206
#The below code for caculation of two points is Edited by Alexander Volkov, Answered by Salvador Dali 
def distance_between_points(lat1, lon1, lat2, lon2):
    p = pi/180     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) *  (1 - cos((lon2 - lon1) * p)) / 2
    
    return 12742 * asin(sqrt(a))
distance_for_points = np.vectorize(distance_between_points)

In [9]:
def distance(output1):
    output1['distance']=distance_for_points(output1['lat'],output1['lon'],output1['lat'].shift(periods=-1),output1['lon'].shift(periods=-1));
    dis = output1['distance'].sum()
    
    del output1['distance']
    return dis * 1000

In [10]:
def output_gpx(points, output_filename):
    """
    Output a GPX file with latitude and longitude from the points DataFrame.
    """
    from xml.dom.minidom import getDOMImplementation
    def append_trkpt(pt, trkseg, doc):
        trkpt = doc.createElement('trkpt')
        trkpt.setAttribute('lat', '%.8f' % (pt['lat']))
        trkpt.setAttribute('lon', '%.8f' % (pt['lon']))
        trkseg.appendChild(trkpt)
    
    doc = getDOMImplementation().createDocument(None, 'gpx', None)
    trk = doc.createElement('trk')
    doc.documentElement.appendChild(trk)
    trkseg = doc.createElement('trkseg')
    trk.appendChild(trkseg)
    
    points.apply(append_trkpt, axis=1,   trkseg=trkseg, doc=doc)
    
    with open(output_filename, 'w') as fh:
        doc.writexml(fh, indent=' ')

In [19]:
print("Total distance travled: " + str(distance(data)) + " meter")
output_gpx(data, 'out.gpx')

Total distance travled: 10659.618426367668 meter
